In [1]:
import cPickle as pk
PK_FPATH = 'data/TREC-DRMM-preprocessed-0125.pk'

In [2]:
with open(PK_FPATH) as f:
    data_pickle = pk.load(f)

In [3]:
QUERIES = data_pickle['QUERIES']
IDFs = data_pickle['IDFs']
candidates = data_pickle['candidates']
relevance = data_pickle['relevance']
qid_docid2histvec = data_pickle['qid_docid2histvec']
instances = data_pickle['instances']

In [4]:
from tqdm import tqdm 
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, InputLayer, Flatten, Input, Merge, merge, Reshape
import keras.backend as K
from keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf
import numpy as np 
import pydot
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot

N_HISTBINS = 30 
BATCH_SZ = 64
N_EPOCH = 50
LOGDIR = './logs/DRMM_0125'

# generator for model training
def batch_generator(idx_pairs, batch_size=BATCH_SZ): 
    # ** parameter `idx_pairs` is list of tuple (qid, pos_docid, neg_docid)**
    np.random.shuffle(idx_pairs)
    batches_pre_epoch = len(idx_pairs) // batch_size
    samples_per_epoch = batches_pre_epoch * batch_size # make samples_per_epoch a multiple of batch size
    counter = 0
    y_true_batch_dummy = np.ones((batch_size))
    while 1:
        idx_batch = idx_pairs[batch_size*counter: min(samples_per_epoch, batch_size*(counter+1))]
        idfs_batch, pos_batch, neg_batch = [], [], []
        for qid, pos_docid, neg_docid in idx_batch:
            idfs_batch.append(IDFs[qid])
            pos_batch.append(qid_docid2histvec[(qid,pos_docid)].reshape(-1,30))
            neg_batch.append(qid_docid2histvec[(qid,neg_docid)].reshape(-1,30))
        idfs_batch, pos_batch, neg_batch = map(np.array, [idfs_batch, pos_batch, neg_batch])
#         print idfs_batch.shape, pos_batch.shape, neg_batch.shape
        counter += 1
        if (counter >= batches_pre_epoch):
            np.random.shuffle(idx_pairs)
            counter=0
        yield [idfs_batch, pos_batch, neg_batch], y_true_batch_dummy

def get_idx_pairs(qids, instances):
    idx_pairs = []
    for qid in qids:
        for posid, negid in instances[qid]:
            idx_pairs.append( (qid,posid, negid) )
    return idx_pairs

def shuffle_weights(model, weights=None):
    """Randomly permute the weights in `model`, or the given `weights`.
    This is a fast approximation of re-initializing the weights of a model.
    Assumes weights are distributed independently of the dimensions of the weight tensors
      (i.e., the weights have the same distribution along each dimension).
    :param Model model: Modify the weights of the given model.
    :param list(ndarray) weights: The model's weights will be replaced by a random permutation of these weights.
      If `None`, permute the model's current weights.
    """
    if weights is None:
        weights = model.get_weights()
    weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
    model.set_weights(weights)

def TREC_output(scoring_model, qid, run_name = 'my_run', fpath = None):
    res = [] # list of (score, pmcid) tuples
    for docid in candidates[qid]:
        input_idf = IDFs[qid].reshape((-1,len(QUERIES[qid])))
        input_hist = qid_docid2histvec[(qid,docid)]
        score = scoring_model.predict([input_idf, input_hist])[0]
        res.append( (score, docid) )
    res = sorted(res, reverse=True)
    fout = sys.stdout if fpath==None else open(fpath, 'a')
    for rank, (score, docid) in enumerate(res[:2000]):
        print >>fout, '%d  Q0  %s  %d  %f  %s' % (qid, docid, rank, score, run_name)

_callbacks = [ EarlyStopping(monitor='val_loss', patience=2),
               TensorBoard(log_dir=LOGDIR, histogram_freq=0, write_graph=False) ]

def KFold(qids, fpath, ranking_model, scoring_model, instances, K = 5, run_name = 'my_run',  batch_size=BATCH_SZ):
    open(fpath,'w').close() # clear previous content in file 
    np.random.seed(0)
    np.random.shuffle(qids)
    fold_sz = len(qids) / K
    for fold in xrange(K):
        print 'fold %d' % fold, 
        val_start, val_end = fold*fold_sz, (fold+1)*fold_sz
        qids_val = qids[val_start:val_end] # train/val queries for each fold 
        qids_train = qids[:val_start] + qids[val_end:]
        print qids_val
        idx_pairs_train = get_idx_pairs(qids_train,instances)
        idx_pairs_val = get_idx_pairs(qids_val,instances)
        
        shuffle_weights(ranking_model, ranking_model.initial_weights) # reset model parameters
        ranking_model.fit_generator( batch_generator(idx_pairs_train, batch_size=batch_size), # train model 
                    samples_per_epoch = len(idx_pairs_train)//batch_size*batch_size,
                    nb_epoch = N_EPOCH,
                    validation_data=batch_generator(idx_pairs_val, batch_size=batch_size),
                    nb_val_samples=len(idx_pairs_val)//batch_size*batch_size, 
                    callbacks = _callbacks)
        print 'fold %d complete, outputting to %s...' % (fold, fpath)
        for qid in qids_val:
            TREC_output(scoring_model, qid, run_name = run_name, fpath = fpath)

Using TensorFlow backend.


In [122]:
# change to only 5 epochs?
def KFold(qids, fpath, ranking_model, scoring_model, instances, K = 5, run_name = 'my_run',  batch_size=BATCH_SZ):
    open(fpath,'w').close() # clear previous content in file 
    np.random.seed(0)
    np.random.shuffle(qids)
    fold_sz = len(qids) / K
    for fold in xrange(K):
        print 'fold %d' % fold, 
        val_start, val_end = fold*fold_sz, (fold+1)*fold_sz
        qids_val = qids[val_start:val_end] # train/val queries for each fold 
        qids_train = qids[:val_start] + qids[val_end:]
        print qids_val
        idx_pairs_train = get_idx_pairs(qids_train,instances)
        idx_pairs_val = get_idx_pairs(qids_val,instances)
        
        shuffle_weights(ranking_model, ranking_model.initial_weights) # reset model parameters
        ranking_model.fit_generator( batch_generator(idx_pairs_train, batch_size=batch_size), # train model 
                    samples_per_epoch = len(idx_pairs_train)//batch_size*batch_size,
                    nb_epoch = 5,
                    validation_data=batch_generator(idx_pairs_val, batch_size=batch_size),
                    nb_val_samples=len(idx_pairs_val)//batch_size*batch_size, )
        print 'fold %d complete, outputting to %s...' % (fold, fpath)
        for qid in qids_val:
            TREC_output(scoring_model, qid, run_name = run_name, fpath = fpath)

scoring_model, ranking_model = gen_DRMM_model(58)
KFold(qids = QUERIES.keys(), fpath = 'data/trec-output/0130_summary_5fold_padded_5epoch.rankedlist', 
  ranking_model=ranking_model, scoring_model=scoring_model, instances = instances)

fold 0 [3, 29, 14, 11, 27, 25]
Epoch 1/5
125120/125120 [==============================] - 21s - loss: 0.0827 - ranking_acc: 0.6221 - val_loss: 0.0905 - val_ranking_acc: 0.5774
Epoch 2/5
125120/125120 [==============================] - 16s - loss: 0.0799 - ranking_acc: 0.6406 - val_loss: 0.0888 - val_ranking_acc: 0.5907
Epoch 3/5
125120/125120 [==============================] - 15s - loss: 0.0788 - ranking_acc: 0.6477 - val_loss: 0.0874 - val_ranking_acc: 0.6013
Epoch 4/5
125120/125120 [==============================] - 16s - loss: 0.0779 - ranking_acc: 0.6539 - val_loss: 0.0865 - val_ranking_acc: 0.6002
Epoch 5/5
125120/125120 [==============================] - 17s - loss: 0.0771 - ranking_acc: 0.6590 - val_loss: 0.0865 - val_ranking_acc: 0.6039
fold 0 complete, outputting to data/trec-output/0130_summary_5fold_padded_5epoch.rankedlist...
fold 1 [28, 12, 18, 23, 6, 17]
Epoch 1/5
123904/123904 [==============================] - 17s - loss: 0.0905 - ranking_acc: 0.5749 - val_loss: 0.0853

In [5]:
N_PAIRS_PER_QUERY = 8000 # the smallest query (22) have 8816 possible pairs 
from numpy.random import choice 
from collections import defaultdict
instances2 = {} # mapping qid to list, instances[qid] = list (pos_docid, neg_docid) pairs for qid, 

np.random.seed(1)
for qid in QUERIES.keys():
    rel_scores = defaultdict(list) # mapping a rel score to list of docids
    for docid in candidates[qid]:
        rel = relevance[(qid,docid)]
        rel_scores[rel].append(docid)
    scores = sorted( rel_scores.keys(), reverse=True ) # scores are sorted in desc order
    print 'scores =',scores, 
    
    all_instances = []
    for i in xrange(len(scores)): 
        pos_score = scores[i]
        for j in xrange(i+1, len(scores)): 
            neg_score = scores[j]
            for posid in rel_scores[pos_score]:
                for negid in rel_scores[neg_score]: 
                    all_instances.append( (posid, negid) )
    
    instances_for_q = []
    for i in choice(len(all_instances), N_PAIRS_PER_QUERY, replace=False):
        instances_for_q.append(all_instances[i])
    
    print 'got %d instances out of %d for query %d' % (len(instances_for_q), len(all_instances), qid)
    instances2[qid] = instances_for_q

scores = [2, 1, 0] got 8000 instances out of 154789 for query 1
scores = [2, 1, 0] got 8000 instances out of 39788 for query 2
scores = [2, 1, 0] got 8000 instances out of 177911 for query 3
scores = [2, 1, 0] got 8000 instances out of 23625 for query 4
scores = [2, 1, 0] got 8000 instances out of 123781 for query 5
scores = [2, 1, 0] got 8000 instances out of 95268 for query 6
scores = [2, 1, 0] got 8000 instances out of 68628 for query 7
scores = [2, 1, 0] got 8000 instances out of 397930 for query 8
scores = [2, 1, 0] got 8000 instances out of 118530 for query 9
scores = [2, 0] got 8000 instances out of 20501 for query 10
scores = [2, 1, 0] got 8000 instances out of 238181 for query 11
scores = [2, 1, 0] got 8000 instances out of 114899 for query 12
scores = [2, 1, 0] got 8000 instances out of 158044 for query 13
scores = [2, 1, 0] got 8000 instances out of 120987 for query 14
scores = [2, 1, 0] got 8000 instances out of 65894 for query 15
scores = [2, 1, 0] got 8000 instances out o

In [6]:
import DRMM
reload(DRMM)
from DRMM import gen_DRMM_model

In [7]:
def tune_ffwd(feed_forward, suffix = '', instances = instances):
    scoring_model, ranking_model = gen_DRMM_model(58, feed_forward)
    KFold(qids = QUERIES.keys(), fpath = 'data/trec-output/0125_summary_5fold_%s.rankedlist' % suffix, 
      ranking_model=ranking_model, scoring_model=scoring_model, instances = instances)
    KFold(qids = QUERIES.keys(), fpath = 'data/trec-output/0125_summary_LOO_%s.rankedlist' % suffix, K=30, 
      ranking_model=ranking_model, scoring_model=scoring_model, instances = instances)

In [8]:
ffwd_3layer = Sequential(
    [Dense(input_dim= N_HISTBINS, output_dim=10, activation='relu'),
     Dense(input_dim= N_HISTBINS, output_dim=5, activation='relu'),
     Dense(output_dim=1, activation='tanh'),
     ], 
    name='ffwd_3layer')
# tune_ffwd(ffwd_3layer, 'ffwd3layer')

In [9]:
ffwd_4layer = Sequential(
    [Dense(input_dim= N_HISTBINS, output_dim=10, activation='relu'),
     Dense(input_dim= N_HISTBINS, output_dim=6, activation='relu'),
     Dense(input_dim= N_HISTBINS, output_dim=3, activation='relu'),
     Dense(output_dim=1, activation='tanh'),
     ], 
    name='ffwd_4layer')
# tune_ffwd(ffwd_4layer, 'ffwd4layer')

In [ ]:
tune_ffwd(ffwd_3layer, 'ffwd3layer_unifsampling', instances=instances2)

In [ ]:
tune_ffwd(ffwd_4layer, 'ffwd4layer_unifsampling', instances=instances2)

fold 0 [3, 29, 14, 11, 27, 25]
Epoch 1/50
192000/192000 [==============================] - 36s - loss: 0.0794 - ranking_acc: 0.6486 - val_loss: 0.0781 - val_ranking_acc: 0.6546
Epoch 2/50
192000/192000 [==============================] - 37s - loss: 0.0723 - ranking_acc: 0.6855 - val_loss: 0.0778 - val_ranking_acc: 0.6593
Epoch 3/50
192000/192000 [==============================] - 36s - loss: 0.0707 - ranking_acc: 0.6922 - val_loss: 0.0787 - val_ranking_acc: 0.6571
Epoch 4/50
192000/192000 [==============================] - 32s - loss: 0.0699 - ranking_acc: 0.6966 - val_loss: 0.0782 - val_ranking_acc: 0.6583
Epoch 5/50
192000/192000 [==============================] - 31s - loss: 0.0692 - ranking_acc: 0.7003 - val_loss: 0.0784 - val_ranking_acc: 0.6585
fold 0 complete, outputting to data/trec-output/0125_summary_5fold_ffwd4layer_unifsampling.rankedlist...
fold 1 [28, 12, 18, 23, 6, 17]
Epoch 1/50
192000/192000 [==============================] - 31s - loss: 0.0968 - ranking_acc: 0.5345 - 

### best result so far: 3 layer uniform sampling, (map=0.19)

# Testing the model implementation

In [51]:
hist_pos = qid_docid2histvec[(1, '1131908')]
hist_neg = qid_docid2histvec[(1, '107838')]

In [52]:
scoring_model.predict([IDFs[1].reshape(1, 58), hist_pos])

array([ 0.91425121], dtype=float32)

In [53]:
scoring_model.predict([IDFs[1].reshape(1, 58), hist_neg])

array([ 0.8934778], dtype=float32)

In [54]:
two_score_model.predict([IDFs[1].reshape(1, 58), hist_pos, hist_neg])

[array([ 0.91425121], dtype=float32), array([ 0.8934778], dtype=float32)]

In [55]:
ranking_model.predict([IDFs[1].reshape(1, 58), hist_pos, hist_neg])

array([ 0.02077341], dtype=float32)

# Variable length DRMM implementation

In [43]:
from lxml import etree
topic_tree = etree.parse('data/topics2016.xml')
def get_topic(i):# returns the summary string of the ith topic
    summary = topic_tree.xpath('//topic[@number="%d"]/summary/text()'%i)[0]
    return str(summary).lower().strip()

from nltk.corpus import stopwords
stopwds = set(stopwords.words('english'))

QUERIES_ORIGIN = {} # NOT padded queries
for qid in xrange(1,31):
    query = get_topic(qid)
    q = [wd for wd in query.split() if wd not in stopwds]
    QUERIES_ORIGIN[qid] = q

In [45]:
print map(len, QUERIES_ORIGIN.values())

[8, 15, 37, 22, 15, 25, 24, 21, 17, 18, 17, 17, 35, 16, 55, 22, 15, 29, 36, 23, 36, 9, 12, 13, 23, 58, 20, 14, 28, 11]


In [51]:
def batch_generator_origin(idx_pairs, qid, batch_size=BATCH_SZ): 
    # ** parameter `idx_pairs` is list of tuple (qid, pos_docid, neg_docid)**
    np.random.shuffle(idx_pairs)
    qlen = len(QUERIES_ORIGIN[qid])
    batches_pre_epoch = len(idx_pairs) // batch_size
    samples_per_epoch = batches_pre_epoch * batch_size # make samples_per_epoch a multiple of batch size
    counter = 0
    y_true_batch_dummy = np.ones((batch_size))
    while 1:
        idx_batch = idx_pairs[batch_size*counter: min(samples_per_epoch, batch_size*(counter+1))]
        idfs_batch, pos_batch, neg_batch = [], [], []
        for qid, pos_docid, neg_docid in idx_batch:
            idfs_batch.append(IDFs[qid][:qlen])
            pos_batch.append(qid_docid2histvec[(qid,pos_docid)][:,:qlen,:].reshape(-1,30))
            neg_batch.append(qid_docid2histvec[(qid,neg_docid)][:,:qlen,:].reshape(-1,30))
        idfs_batch, pos_batch, neg_batch = map(np.array, [idfs_batch, pos_batch, neg_batch])
#         print idfs_batch.shape, pos_batch.shape, neg_batch.shape
        counter += 1
        if (counter >= batches_pre_epoch):
            np.random.shuffle(idx_pairs)
            counter=0
        yield [idfs_batch, pos_batch, neg_batch], y_true_batch_dummy

In [128]:
MEM = {}
def MODELS(qid):
    if qid in MEM: return MEM[qid]
    else: 
        res = gen_DRMM_model(len(QUERIES_ORIGIN[qid]))
        MEM[qid] = res
        return res

def train_for_query(qid, instances=instances, batch_size = BATCH_SZ):
#     print 'training on query %d...' % qid,
    drmm_scoring, drmm_ranking = MODELS(qid)
    idx_pairs = [(qid,posid, negid) for (posid, negid) in instances[qid]]
    drmm_ranking.fit_generator( batch_generator_origin(idx_pairs, qid), verbose=0,
                    samples_per_epoch = len(idx_pairs)//batch_size*batch_size,
                    nb_epoch = 1 )

def test_on_query(qid, instances=instances):
    print 'testing on query %d...' % qid,
    drmm_scoring, drmm_ranking = MODELS(qid)
    idx_pairs = [(qid,posid, negid) for (posid, negid) in instances[qid]]
    r = drmm_ranking.evaluate_generator(batch_generator_origin(idx_pairs, qid), 
                                   len(idx_pairs) )
    print 'loss = %.4f, ranking_acc = %.4f' % tuple(r)

import sys
def output_for_query(qid, fpath=None, run_name='my_run'):
    print 'outputting results for query %d' % qid 
    qlen = len(QUERIES_ORIGIN[qid])
    drmm_scoring, drmm_ranking = MODELS(qid)
    res = [] # list of (score, pmcid) tuples
    for docid in candidates[qid]:
        input_idf = IDFs[qid][:qlen].reshape((-1,qlen))
        input_hist = qid_docid2histvec[(qid,docid)][:,:qlen,:]
        score = drmm_scoring.predict([input_idf, input_hist])[0]
        res.append( (score, docid) )
    res = sorted(res, reverse=True)
    fout = sys.stdout if fpath==None else open(fpath, 'a')
    for rank, (score, docid) in enumerate(res[:2000]):
        print >>fout, '%d  Q0  %s  %d  %f  %s' % (qid, docid, rank, score, run_name)

In [129]:
import DRMM
reload(DRMM)
from DRMM import gen_DRMM_model, feed_forward, gating
ffwd_initial_weights = feed_forward.get_weights()
gating_init_weights = gating.get_weights()

def reset_weights(): # reset model parameters
    shuffle_weights(feed_forward, ffwd_initial_weights) 
    shuffle_weights(gating, gating_init_weights)

def KFold_varlen(qids, fpath, instances=instances, K = 5, N_EPOCH = 5):
    open(fpath,'w').close() # clear previous content in file 
    np.random.seed(0)
    np.random.shuffle(qids)
    fold_sz = len(qids) / K
    for fold in xrange(K):
        print '##### fold %d #####' % fold, 
        val_start, val_end = fold*fold_sz, (fold+1)*fold_sz
        qids_val = qids[val_start:val_end] # train/val queries for each fold 
        qids_train = qids[:val_start] + qids[val_end:]
        print qids_val
        
        reset_weights()
        for epoch in xrange(N_EPOCH):
            print '### epoch %d ###' % epoch
            for qtrain in tqdm(qids_train):
                train_for_query(qtrain, instances)
            for qtest in qids_val: 
                test_on_query(qtest, instances)
        
        print 'fold %d complete, outputting to %s...' % (fold, fpath)
        for qid in qids_val:
            output_for_query(qid, fpath = fpath)

In [130]:
KFold_varlen(QUERIES_ORIGIN.keys(), fpath = 'data/trec-output/0130_summary_5fold_varlen_5epoch_uniform.rankedlist', 
            instances = instances2, K=5, N_EPOCH=5)

##### fold 0 ##### [3, 29, 14, 11, 27, 25]


  0%|          | 0/24 [00:00<?, ?it/s]

### epoch 0 ###


100%|██████████| 24/24 [07:34<00:00, 17.19s/it]


testing on query 3... loss = 0.0957, ranking_acc = 0.5546
testing on query 29... loss = 0.0562, ranking_acc = 0.7621
testing on query 14... loss = 0.0887, ranking_acc = 0.6300
testing on query 11... loss = 0.1057, ranking_acc = 0.5424
testing on query 27... loss = 0.0694, ranking_acc = 0.6944
testing on query 25...

...

testing on query 26... loss = 0.0716, ranking_acc = 0.7153
testing on query 4... loss = 0.0829, ranking_acc = 0.6529
testing on query 1... loss = 0.0689, ranking_acc = 0.7005
testing on query 22... loss = 0.0986, ranking_acc = 0.5288
testing on query 16... loss = 0.0994, ranking_acc = 0.5101
testing on query 13...

  0%|          | 0/24 [00:00<?, ?it/s]

 loss = 0.0909, ranking_acc = 0.5731
### epoch 4 ###


100%|██████████| 24/24 [00:16<00:00,  1.51it/s]


testing on query 26... loss = 0.0714, ranking_acc = 0.7129
testing on query 4... loss = 0.0823, ranking_acc = 0.6571
testing on query 1... loss = 0.0698, ranking_acc = 0.6956
testing on query 22... loss = 0.0966, ranking_acc = 0.5507
testing on query 16... loss = 0.0996, ranking_acc = 0.5112
testing on query 13... loss = 0.0906, ranking_acc = 0.5751
fold 4 complete, outputting to data/trec-output/0130_summary_5fold_varlen_5epoch_uniform.rankedlist...
outputting results for query 26
outputting results for query 4
outputting results for query 1
outputting results for query 22
outputting results for query 16
outputting results for query 13


In [131]:
KFold_varlen(QUERIES_ORIGIN.keys(), fpath = 'data/trec-output/0130_summary_5fold_varlen_10epoch_uniform.rankedlist', 
            instances = instances2, K=5, N_EPOCH=10)

  0%|          | 0/24 [00:00<?, ?it/s]

##### fold 0 ##### [3, 29, 14, 11, 27, 25]
### epoch 0 ###


100%|██████████| 24/24 [00:16<00:00,  1.46it/s]


testing on query 3... loss = 0.0980, ranking_acc = 0.5182
testing on query 29... loss = 0.0901, ranking_acc = 0.5954
testing on query 14... loss = 0.0835, ranking_acc = 0.6631
testing on query 11... loss = 0.0975, ranking_acc = 0.5178
testing on query 27... loss = 0.1021, ranking_acc = 0.4519
testing on query 25...

  0%|          | 0/24 [00:00<?, ?it/s]

...

testing on query 26... loss = 0.0708, ranking_acc = 0.7212
testing on query 4... loss = 0.0718, ranking_acc = 0.7288
testing on query 1... loss = 0.0704, ranking_acc = 0.6947
testing on query 22... loss = 0.0973, ranking_acc = 0.5380
testing on query 16... loss = 0.0973, ranking_acc = 0.5364
testing on query 13...

  0%|          | 0/24 [00:00<?, ?it/s]

 loss = 0.0897, ranking_acc = 0.5776
### epoch 9 ###


100%|██████████| 24/24 [00:16<00:00,  1.47it/s]


testing on query 26... loss = 0.0706, ranking_acc = 0.7230
testing on query 4... loss = 0.0723, ranking_acc = 0.7266
testing on query 1... loss = 0.0699, ranking_acc = 0.6979
testing on query 22... loss = 0.0967, ranking_acc = 0.5435
testing on query 16... loss = 0.0975, ranking_acc = 0.5311
testing on query 13... loss = 0.0898, ranking_acc = 0.5777
fold 4 complete, outputting to data/trec-output/0130_summary_5fold_varlen_10epoch_uniform.rankedlist...
outputting results for query 26
outputting results for query 4
outputting results for query 1
outputting results for query 22
outputting results for query 16
outputting results for query 13


In [132]:
KFold_varlen(QUERIES_ORIGIN.keys(), fpath = 'data/trec-output/0130_summary_LOO_varlen_5epoch_uniform.rankedlist', 
            instances = instances2, K=30, N_EPOCH=5)

  0%|          | 0/29 [00:00<?, ?it/s]

##### fold 0 ##### [3]
### epoch 0 ###


100%|██████████| 29/29 [00:19<00:00,  1.45it/s]


testing on query 3...

  0%|          | 0/29 [00:00<?, ?it/s]

...

testing on query 13...

  0%|          | 0/29 [00:00<?, ?it/s]

 loss = 0.0984, ranking_acc = 0.4953
### epoch 4 ###


100%|██████████| 29/29 [00:19<00:00,  1.49it/s]


testing on query 13... loss = 0.0983, ranking_acc = 0.4956
fold 29 complete, outputting to data/trec-output/0130_summary_LOO_varlen_5epoch_uniform.rankedlist...
outputting results for query 13


In [133]:
KFold_varlen(QUERIES_ORIGIN.keys(), fpath = 'data/trec-output/0130_summary_LOO_varlen_10epoch_uniform.rankedlist', 
            instances = instances2, K=30, N_EPOCH=10)

  0%|          | 0/29 [00:00<?, ?it/s]

##### fold 0 ##### [3]
### epoch 0 ###


100%|██████████| 29/29 [00:19<00:00,  1.44it/s]


testing on query 3...

  0%|          | 0/29 [00:00<?, ?it/s]

...

testing on query 13...

  0%|          | 0/29 [00:00<?, ?it/s]

 loss = 0.0829, ranking_acc = 0.6215
### epoch 9 ###


100%|██████████| 29/29 [00:19<00:00,  1.47it/s]


testing on query 13... loss = 0.0825, ranking_acc = 0.6229
fold 29 complete, outputting to data/trec-output/0130_summary_LOO_varlen_10epoch_uniform.rankedlist...
outputting results for query 13


In [125]:
KFold_varlen(QUERIES_ORIGIN.keys(), fpath = 'data/trec-output/0130_summary_LOO_varlen_5epoch.rankedlist', 
                 K = 30, N_EPOCH=5 )

##### fold 0 ##### [3]


  0%|          | 0/29 [00:00<?, ?it/s]

### epoch 0 ###
Epoch 1/1
3520/3520 [==============================] - 4s - loss: 0.0895 - ranking_acc: 0.5946     


  3%|▎         | 1/29 [00:08<04:11,  8.99s/it]

Epoch 1/1
3456/3456 [==============================] - 3s - loss: 0.0761 - ranking_acc: 0.6620     


...

 97%|█████████▋| 28/29 [00:13<00:00,  2.61it/s]

Epoch 1/1
5248/5248 [==============================] - 0s - loss: 0.0989 - ranking_acc: 0.5168     


100%|██████████| 29/29 [00:13<00:00,  2.49it/s]

testing on query 13...

 loss = 0.0973, ranking_acc = 0.5391
fold 29 complete, outputting to data/trec-output/0130_summary_LOO_varlen_5epoch.rankedlist...
outputting results for query 13


In [127]:
KFold_varlen(QUERIES_ORIGIN.keys(), fpath = 'data/trec-output/0130_summary_LOO_varlen_10epoch.rankedlist', 
                 K = 30, N_EPOCH=10 )

  0%|          | 0/29 [00:00<?, ?it/s]

##### fold 0 ##### [3]
### epoch 0 ###


100%|██████████| 29/29 [06:40<00:00, 15.00s/it]


testing on query 3...

  0%|          | 0/29 [00:00<?, ?it/s]

...

testing on query 13...

  0%|          | 0/29 [00:00<?, ?it/s]

 loss = 0.1002, ranking_acc = 0.5052
### epoch 9 ###


100%|██████████| 29/29 [00:13<00:00,  2.60it/s]


testing on query 13... loss = 0.1002, ranking_acc = 0.5089
fold 29 complete, outputting to data/trec-output/0130_summary_LOO_varlen_10epoch.rankedlist...
outputting results for query 13


In [119]:
KFold_varlen(QUERIES_ORIGIN.keys(), fpath = 'data/trec-output/0130_summary_5fold_varlen.rankedlist', K = 5 )

fold 0 [3, 29, 14, 11, 27, 25]
### epoch 0 ###
training on query 28... training on query 12... training on query 18... training on query 23... training on query 6... training on query 17... training on query 9... training on query 15... training on query 24... training on query 21... training on query 2... training on query 30... training on query 7... training on query 5... training on query 19... training on query 20... training on query 10... training on query 8... training on query 26... training on query 4... training on query 1... training on query 22... training on query 16... training on query 13... testing on query 3... loss = 0.0981, ranking_acc = 0.5326
testing on query 29... loss = 0.0554, ranking_acc = 0.7687
testing on query 14... loss = 0.0896, ranking_acc = 0.6188
testing on query 11... loss = 0.1048, ranking_acc = 0.5498
testing on query 27... loss = 0.0574, ranking_acc = 0.7590
testing on query 25... loss = 0.0673, ranking_acc = 0.7306
### epoch 1 ###
training on quer

In [120]:
KFold_varlen(QUERIES_ORIGIN.keys(), fpath = 'data/trec-output/0130_summary_5fold_varlen_10epoch.rankedlist', 
             K = 5, N_EPOCH=10)

fold 0 [3, 29, 14, 11, 27, 25]
### epoch 0 ###
training on query 28... training on query 12... training on query 18... training on query 23... training on query 6... training on query 17... training on query 9... training on query 15... training on query 24... training on query 21... training on query 2... training on query 30... training on query 7... training on query 5... training on query 19... training on query 20... training on query 10... training on query 8... training on query 26... training on query 4... training on query 1... training on query 22... training on query 16... training on query 13... testing on query 3... loss = 0.0994, ranking_acc = 0.5014
testing on query 29... loss = 0.0899, ranking_acc = 0.6018
testing on query 14... loss = 0.0869, ranking_acc = 0.6443
testing on query 11... loss = 0.1006, ranking_acc = 0.5029
testing on query 27... loss = 0.1050, ranking_acc = 0.4141
testing on query 25... loss = 0.0894, ranking_acc = 0.6220
### epoch 1 ###
training on quer

In [93]:
input_idf2 = Input(shape=(8,), name='input_idf2')
gs = gating(input_idf2)

In [90]:
print gating.predict(IDFs[1].reshape(1,-1))

[[  1.08647980e-01   8.66970513e-03   9.70950723e-03   2.63532810e-02
    9.77671891e-02   6.89344853e-02   6.77386999e-01   2.52833311e-03
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08   4.22274447e-08   4.22274447e-08   4.22274447e-08
    4.22274447e-08  